In [1]:
# Evaluation Metric
class MonostateSingleton:
    __shared_state = {}
    def __init__(self):
        self.__dict__ = self.__shared_state

m1 = MonostateSingleton()
m2 = MonostateSingleton()
print(m1) # <__main__.MonostateSingleton object at 0x02C4B0E8>
print(m2) # <__main__.MonostateSingleton object at 0x02C4B118>

m1.a = 1
m2.b = 2
m2.c = 0
m1.a += 1
print(m1.__dict__) # {'a': 1, 'b': 2}
print(m2.__dict__) # {'a': 1, 'b': 2}x03A6B340>

{'a': 2, 'b': 2, 'c': 0}
{'a': 2, 'b': 2, 'c': 0}


In [2]:
m1.a += 1

In [3]:
m1.a += 1

In [4]:
m1.__dict__

{'a': 4, 'b': 2, 'c': 0}

In [5]:
m2.a += 3

In [6]:
m1.__dict__

{'a': 7, 'b': 2, 'c': 0}

In [7]:
total = 867
for i in range(0,total):
    #print(int(total * 0.1))
    if i % int((total*0.1)) ==0:
        print(i)

0
86
172
258
344
430
516
602
688
774
860


In [8]:
#(progress*100) % 10 == 0:

for i in range(0,10):
    #print(i)
    z = i * 0.01
    #print(z)
    print( (z * 100) % 5   )


0.0
1.0
2.0
3.0
4.0
0.0
1.0
2.000000000000001
3.0
4.0


In [9]:
class LazyInstantiation:
    _instance = None
    _shared_data = {"a":0}
    def __init__(self):
        if not LazyInstantiation._instance:
            print('__init__ method called but nothing is created')
        else:
            print('instance already created:', self.getInstance())
 
    @classmethod
    def getInstance(cls):
        if not cls._instance:
            cls._instance = LazyInstantiation()
        return cls._instance
 
    def data_update(self,v):
        self._shared_data["a"] += v

        
# __init__ method called but nothing is created
s = LazyInstantiation()
print(s._instance) # None
# __init__ method called but nothing is created
s1 = LazyInstantiation.getInstance()
s2 = LazyInstantiation.getInstance()
print('s와 s1은 서로 같은 인스턴스인가? -> ',s==s1) # True
print('s1 : ',s1._instance) # <__main__.LazyInstantiation object at 0x03A6B340>
print('s2 : ',s2._instance) # <__main__.LazyInstantiation object at 0x03A6B340>
 
s3 = LazyInstantiation() # instance already created: <__main__.LazyInstantiation object at 0x03A6B340>
print('s3 : ',s3._instance) #  <__main__.LazyInstantiation object at 0x03A6B340>

__init__ method called but nothing is created
None
__init__ method called but nothing is created
s와 s1은 서로 같은 인스턴스인가? ->  False
s1 :  <__main__.LazyInstantiation object at 0x7f9d004cab50>
s2 :  <__main__.LazyInstantiation object at 0x7f9d004cab50>
instance already created: <__main__.LazyInstantiation object at 0x7f9d004cab50>
s3 :  <__main__.LazyInstantiation object at 0x7f9d004cab50>


In [10]:
s.data_update(3)

In [11]:
s._shared_data

{'a': 3}

In [12]:
s.data_update(3)

In [13]:
s._shared_data

{'a': 6}

In [14]:
s3._shared_data

{'a': 6}

In [15]:
s3.data_update(4)

In [16]:
s3._shared_data

{'a': 10}

In [17]:
s._shared_data['a']

10

In [18]:
from multiprocessing import Process, Manager

class Singleton:
    _instance = None

    def __init__(self, value):
        if Singleton._instance is not None:
            raise Exception("This is a singleton class!")
        self.value = value
        Singleton._instance = self

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            raise Exception("Instance is not created yet!")
        return cls._instance

def create_singleton(value, shared_dict):
    singleton = Singleton(value)
    shared_dict['singleton'] = singleton.value

In [19]:
if __name__ == '__main__':
    manager = Manager()
    shared_dict = manager.dict()

    p1 = Process(target=create_singleton, args=(10, shared_dict))
    p2 = Process(target=create_singleton, args=(20, shared_dict))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Singleton value:", shared_dict['singleton'])


Singleton value: 20


In [20]:
import threading

class Singleton:
    _instance = None
    _lock = threading.Lock()
    _data = {
        "a" :[]
    }

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super(Singleton, cls).__new__(cls)
        return cls._instance

# Example usage
singleton1 = Singleton()
singleton2 = Singleton()

print(singleton1 is singleton2)  # This should print True, indicating both are the same instance


True


In [21]:
singleton1._data['a'].append(10)

In [22]:
singleton2._data['a'].append(10)

In [23]:
singleton2._data

{'a': [10, 10]}

- sentinel test

In [24]:
#---

'''
01.22
sentinel2 
64x64 => 224x224 


path note 01.20
holdout => k-fold ! 


'''


#--
import sys
sys.path.append("/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/07.Challenge/01.MapYourCity_HuggingFace")
import map_dataset
import map_train
from models import *

sys.path.append("/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/00.Libs")
import RS_dataset
import RS_models
import RS_utils
#--- torch
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
#--- loss functions
from utils.losses import LabelSmoothCrossEntropy, CrossEntropyLoss
from torch.cuda.amp import GradScaler, autocast
#---
from lightning.fabric import Fabric
from lightning.fabric.loggers import CSVLogger, TensorBoardLogger
from torchmetrics.classification import Accuracy
import pandas as pd 
import os 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import argparse
import yaml 
import timm
import numpy as np 
import time
import wandb
from rich.console import Console
from wandb.integration.lightning.fabric import WandbLogger



#--- argparser
parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, \
                    default='/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/07.Challenge/01.MapYourCity_HuggingFace/configs/finetune_22.yaml')
args = parser.parse_args([])
cfg = argparse.Namespace(**yaml.load(open(args.cfg), Loader=yaml.SafeLoader))

#--- Data 
input_path = "/mnt/hdd/eric/.tmp_ipy/00.Data/Map_Your_City/building-age-dataset/"
train_path = input_path + "train/data/"
test_path = input_path + "test/data/"
train_df = pd.read_csv(input_path + "train/train-set.csv")
test_df = pd.read_csv(input_path + "test/test-set.csv") 

#--- data split 
names_data = sorted( os.listdir(train_path) )

names_label = []
for ID in names_data:
    y = int(open(train_path + ID + '/label.txt', "r").read())
    names_label.append(y)

if cfg.SAMPLE:
    parse_idx = int(len(names_data) * cfg.SAMPLE_PERCENT)
    names_data = names_data[:parse_idx]
    names_label = names_label[:parse_idx]


#-- kfold 

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations
/mnt/hdd/eric/.conda/envs/mapv2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for lightning Fabric 2.1.3.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `lightning-fabric`.


In [25]:
#--- model
model = timm.create_model(
cfg.MODEL,
pretrained=True,
num_classes=cfg.CLASSES_NUM ) 

#--- data config and transform
data_config = timm.data.resolve_model_data_config(model)
data_transform = timm.data.create_transform(**data_config, is_training=False)

print(data_config)

#--- train and valid selection need
train_set = map_dataset.Map_Dataset_v12(names_data,train_path,max_size=data_config['input_size'][1],cfg=cfg,split='train') 

/mnt/hdd/eric/.conda/envs/mapv2/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k)
INFO:timm.models._hub:[timm/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


{'input_size': (3, 448, 448), 'interpolation': 'bicubic', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'crop_pct': 1.0, 'crop_mode': 'squash'}


In [26]:
x1,x2,x3, y,y  = train_set.__getitem__(0)

In [27]:
type(x1)

torch.Tensor

In [28]:
type(x2)

torch.Tensor

In [29]:
type(x3)

torch.Tensor

In [30]:
x3

tensor([[[-1.7583, -1.7583, -1.7583,  ..., -1.1589, -1.1589, -1.1589],
         [-1.7583, -1.7583, -1.7583,  ..., -1.1589, -1.1589, -1.1589],
         [-1.7583, -1.7583, -1.7583,  ..., -1.1589, -1.1589, -1.1589],
         ...,
         [ 0.4508,  0.4508,  0.4508,  ..., -1.1247, -1.1247, -1.1247],
         [ 0.4508,  0.4508,  0.4508,  ..., -1.1247, -1.1247, -1.1247],
         [ 0.4508,  0.4508,  0.4508,  ..., -1.1247, -1.1247, -1.1247]],

        [[-1.2304, -1.2304, -1.2304,  ..., -1.0028, -1.0028, -1.0028],
         [-1.2304, -1.2304, -1.2304,  ..., -1.0028, -1.0028, -1.0028],
         [-1.2304, -1.2304, -1.2304,  ..., -1.0028, -1.0028, -1.0028],
         ...,
         [ 0.5028,  0.5028,  0.5028,  ..., -1.2129, -1.2129, -1.2129],
         [ 0.5028,  0.5028,  0.5028,  ..., -1.2129, -1.2129, -1.2129],
         [ 0.5028,  0.5028,  0.5028,  ..., -1.2129, -1.2129, -1.2129]],

        [[-1.6127, -1.6127, -1.6127,  ..., -1.2641, -1.2641, -1.2641],
         [-1.6127, -1.6127, -1.6127,  ..., -1

In [31]:
# array([[[ 21,  46,  11],
#         [ 17,  41,  10],
#         [ 17,  36,  10],
#         ...,
#         [ 47,  52,  25],
#         [ 42,  53,  24],
#         [ 56,  59,  31]],

#        [[ 13,  30,   8],
#         [  9,  24,   4],
#         [ 12,  20,   5],
#         ...,
#         [ 19,  34,  11],
#         [ 19,  36,  12],
#         [ 33,  48,  19]],

#        [[ 12,  19,   5],
#         [ 12,  19,   5],
#         [ 15,  23,   8],
#         ...,
#         [ 42,  52,  26],
#         [ 40,  52,  26],
#         [ 20,  47,  16]],

#        ...,
# ...
#         [ 52,  57,  31],
#         ...,
#         [ 80,  71,  38],
#         [ 61,  49,  31],
#         [ 58,  47,  28]]], dtype=int16)
# Output is truncated. View as a scrollable element or op